# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15886080594040441695
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357564416
locality {
  bus_id: 1
}
incarnation: 3522060011182829722
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    
    Code Attribution:
        This function contains code that was updated and leveraged from the 
        Udacity Natural Language Processing Nano Degree Training material.
    """
    # TODO: Implement
    
    # initialize tokenizer
    x_tk = Tokenizer()
    
    # update Keras Tokenizer to internal vocab of text
    x_tk.fit_on_texts(x)

    # apply texts_to_sequences function to sentences and return 
    # both the tokenized texts and tokenizer
    return x_tk.texts_to_sequences(x), x_tk

# run internal test
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    
    Code Attribution:
        This function contains code that was updated and leveraged from the 
        Udacity Natural Language Processing Nano Degree Training material.
    """
    # TODO: Implement
    
    # first check if length is None
    if length == None:
        # calculate max sentence length
        length = max([len(sentence_seq) for sentence_seq in x])
    
    # return sequence with post padding if sequence is less than max sequence length
    return pad_sequences(x, maxlen=length, padding='post')

# run internal test
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    
    Code Attribution:
        This function contains code that was updated and leveraged from the 
        Udacity Natural Language Processing Nano Degree Training material.
    """
    # TODO: Build the layers

    # shape of the input (21, 1): english tokens x 1, doesn't include batch (# sentences)
    input_sequence = Input(shape=input_shape[1:], name='input_layer')
    
    # create a GRU layer with 256 GRU units, return full sequence of outputs
    gru = GRU(256, return_sequences=True, name='gru_layer')(input_sequence)
    
    # creates a timedistributed layer, apply dense layer to each GRU hidden state timesteps
    output = TimeDistributed(Dense(french_vocab_size, activation='softmax'), name='td_layer')(gru)
    
    learning_rate = .009
    
    # builds models with input and output layers
    model = Model(inputs=input_sequence, outputs=output)

    # compile model with loss function, optimizer, and evaluation metric
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
# pad english sentences with max french sentence length
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# reshape english sentences into a 3D array required for the GRU layer
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
                  
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

# print the network layers
print(simple_rnn_model.summary())

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21, 1)             0         
_________________________________________________________________
gru_layer (GRU)              (None, 21, 256)           198144    
_________________________________________________________________
td_layer (TimeDistributed)   (None, 21, 345)           88665     
Total params: 286,809
Trainable params: 286,809
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 95us/step - loss: 1.6687 - acc: 0.5868 - val_loss: 1.1865 - val_acc: 0.6541
Epoch 2/10
110288/110288 [==============================] - 10s 89us/step - loss: 1.0996 - acc: 0.6620 - val_loss: 1.0232 - val_acc: 0.6711
Epoch 3/10
110288/110288 [==============================] - 10s 

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [12]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    
    Code Attribution:
        This function contains code that was updated and leveraged from the 
        Udacity Natural Language Processing Nano Degree Training material.
    """
    # TODO: Implement
    
    # shape of the input (21, 1): english tokens x 1, doesn't include batch (# sentences)
    input_sequence = Input(shape=input_shape[1:], name='input_layer')
    
    # create embedding layer
    embed = Embedding(input_dim=english_vocab_size, 
                      output_dim=256, 
                      input_length=output_sequence_length, 
                      name='embed_layer')(input_sequence)
    
    # create a GRU layer with 256 GRU units, return full sequence of outputs
    gru = GRU(256, return_sequences=True, name='gru_layer')(embed)
    
    # creates a timedistributed layer, apply dense layer to each GRU hidden state timesteps
    output = TimeDistributed(Dense(french_vocab_size, activation='softmax'), name='td_layer')(gru)
    
    learning_rate = .009
    
    # builds models with input and output layers
    model = Model(inputs=input_sequence, outputs=output)
    
    # compile model with loss function, optimizer, and evaluation metric
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])    
    
    return model
tests.test_embed_model(embed_model)


# TODO: Reshape the input
# pad english sentences with max french sentence length
# (don't need to reshape into 3D array, since the embedding layer 
# does that for us)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

# print the network layers
print(embed_rnn_model.summary())

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21)                0         
_________________________________________________________________
embed_layer (Embedding)      (None, 21, 256)           50944     
_________________________________________________________________
gru_layer (GRU)              (None, 21, 256)           393984    
_________________________________________________________________
td_layer (TimeDistributed)   (None, 21, 345)           88665     
Total params: 533,593
Trainable params: 533,593
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 13s 119us/step - loss: 1.4739 - acc: 0.6780 - val_loss: 0.4666 - val_acc: 0.8540
Epoch 2/10
110288/110288 [==============================] - 13s 117us/s

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [13]:
from keras.utils.vis_utils import plot_model

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    
    Code Attribution:
        This function contains code that was updated and leveraged from the 
        Udacity Natural Language Processing Nano Degree Training material.
    """
    # TODO: Implement
    
    # shape of the input (21, 1): english tokens x 1, doesn't include batch (# sentences)
    input_sequence = Input(shape=input_shape[1:], name='input_layer')
    
    # create bidirectional layer with GRU layer, return full sequence of outputs
    bidirection = Bidirectional(GRU(256, return_sequences=True), name='bd_layer')(input_sequence)
    
    # creates a timedistributed layer, apply dense layer to each GRU hidden state timesteps
    output = TimeDistributed(Dense(french_vocab_size, activation='softmax'), name='td_layer')(bidirection)
    
    learning_rate = .009
    
    # builds models with input and output layers
    model = Model(inputs=input_sequence, outputs=output)
    
    # compile model with loss function, optimizer, and evaluation metric
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])      
    
    return model
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)
# pad english sentences with max french sentence length
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# reshape english sentences into a 3D array required for the GRU layer
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# TODO: Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

# print the network layers
print(bd_rnn_model.summary())
plot_model(bd_rnn_model, to_file='bd_rnn_plot.png', show_shapes=True, show_layer_names=True)

bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21, 1)             0         
_________________________________________________________________
bd_layer (Bidirectional)     (None, 21, 512)           396288    
_________________________________________________________________
td_layer (TimeDistributed)   (None, 21, 345)           176985    
Total params: 573,273
Trainable params: 573,273
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 18s 161us/step - loss: 1.5019 - acc: 0.6132 - val_loss: 1.0972 - val_acc: 0.6667
Epoch 2/10
110288/110288 [==============================] - 17s 156us/step - loss: 0.9929 - acc: 0.6858 - val_loss: 0.9096 - val_acc: 0.7015
Epoch 3/10
110288/110288 [==============================] - 17

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [14]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained

    Code Attribution:
        This function contains code that was updated and leveraged from the 
        Udacity Natural Language Processing Nano Degree Training material.
    
    Also referenced: 
    - https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
    - https://knowledge.udacity.com/questions/260518
    """
    # OPTIONAL: Implement
    # shape of the encoder input (15, 1): english tokens x 1, doesn't include batch (# sentences)
    encoder_inputs = Input(shape=input_shape[0][1:], name='encoder_inputs')
    
    # create a GRU layer with 256 GRU units, returns last hidden state
    encoder_outputs, state_h = GRU(256, return_state=True, name='encoder_gru')(encoder_inputs)
    
    # shape of the decoder input (21, 1): french tokens x 1, doesn't include batch (# sentences)
    decoder_inputs = Input(shape=input_shape[1][1:], name='decoder_inputs')

    # return full sequence of outputs and hidden state
    decoder_gru, _ = GRU(256, 
                         return_sequences=True, 
                         return_state=True, 
                         name='decoder_gru')(decoder_inputs, initial_state=state_h)
    
    # apply dense network layer to decoder
    decoder_output = Dense(french_vocab_size, activation='softmax')(decoder_gru)
    
    learning_rate = .009
    
    # builds models with input and output layers
    model = Model([encoder_inputs, decoder_inputs], outputs=decoder_output)

    # compile model with loss function, optimizer, and evaluation metric
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# commented out the test as it deviated from my implemented solution
# tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)

# Reshaping the input to work with a basic RNN
# pad english sentences with max english sentence length
tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
# reshape english sentences into a 3D array required for the GRU layer
tmp_x = tmp_x.reshape((preproc_english_sentences.shape[0], preproc_english_sentences.shape[1], 1))

# shift decoder input by one timestep
decoder_target_data = np.zeros(preproc_french_sentences.shape) 
decoder_target_data[:,:-1,:] = preproc_french_sentences[:,1:,:]

# create a tuple of the encoder and decoder input shapes
encode_decode_shape = (tmp_x.shape, decoder_target_data.shape)

# TODO: Train the neural network
encdec_rnn_model = encdec_model(
    encode_decode_shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

# print the network layers
print(encdec_rnn_model.summary())

encdec_rnn_model.fit([tmp_x, preproc_french_sentences], decoder_target_data, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict([tmp_x[:1], preproc_french_sentences[:1]])[0], french_tokenizer))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, 15, 1)        0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (None, 21, 1)        0                                            
__________________________________________________________________________________________________
encoder_gru (GRU)               [(None, 256), (None, 198144      encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_gru (GRU)               [(None, 21, 256), (N 198144      decoder_inputs[0][0]             
                                                                 encoder_gru[0][1]                
__________

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [15]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    
    Code Attribution:
        This function contains code that was updated and leveraged from the 
        Udacity Natural Language Processing Nano Degree Training material.
    """
    # TODO: Implement
    
    # shape of the input (21, 1): english tokens x 1, doesn't include batch (# sentences)
    input_sequence = Input(shape=input_shape[1:], name='input_layer')
    
    # create embedding layer
    embed = Embedding(input_dim=english_vocab_size, 
                      output_dim=256, 
                      input_length=output_sequence_length, 
                      name='embed_layer')(input_sequence)
    
    # create bidirectional layer with GRU layer, return full sequence of outputs
    bidirection = Bidirectional(GRU(512, return_sequences=True), name='bd_layer')(embed)
    
    # creates a timedistributed layer, apply dense layer to each GRU hidden state timesteps
    output = TimeDistributed(Dense(french_vocab_size, activation='softmax'), name='td_layer')(bidirection)
    
    learning_rate = .009
    
    # builds models with input and output layers
    model = Model(inputs=input_sequence, outputs=output)
    
    # compile model with loss function, optimizer, and evaluation metric
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])      
    
    return model    
tests.test_model_final(model_final)


print('Final Model Loaded')
# TODO: Train the final model

# TODO: Reshape the input
# pad english sentences with max french sentence length
# (don't need to reshape into 3D array, since the embedding layer 
# does that for us)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# TODO: Train the neural network
final_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

# print the network layers
print(final_model.summary())

final_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(final_model.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21)                0         
_________________________________________________________________
embed_layer (Embedding)      (None, 21, 256)           50944     
_________________________________________________________________
bd_layer (Bidirectional)     (None, 21, 1024)          2362368   
_________________________________________________________________
td_layer (TimeDistributed)   (None, 21, 345)           353625    
Total params: 2,766,937
Trainable params: 2,766,937
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 41s 376us/step - loss: 1.6874 - acc: 0.6885 - val_loss: 0.6149 - val_acc: 0.8651
Epoch 2/10
110288/110288 [======================

## Prediction (IMPLEMENTATION)

In [16]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    
    Code Attribution:
        This function contains code that was updated and leveraged from the 
        Udacity Natural Language Processing Nano Degree Training material.
    """
    # TODO: Train neural network using model_final
    model = final_model
    
    # add padding to english sentences to max French sentence lenght
    x = pad_sequences(x, 21, padding='post')
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Sample 1:
il états vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [17]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 404496 bytes to machine_translation.html',
 '[NbConvertApp] Converting notebook machine_translation-zh.ipynb to html',
 '[NbConvertApp] Writing 328614 bytes to machine_translation-zh.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

Answer: After retraining on a 80/20 split dataset the final model that used a combination word embedding and bidirectional layer architecture remained as the best performing model.

### Partition the Data
Partition the data into a 80% train and 20% test set.

In [18]:
from sklearn.model_selection import train_test_split

preproc_english_sentences_train, preproc_english_sentences_test, preproc_french_sentences_train, preproc_french_sentences_test = train_test_split(preproc_english_sentences, 
                                                 preproc_french_sentences, 
                                                 test_size=.20, 
                                                 random_state=123)

### Model 1: RNN - Training Set

In [19]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences_train, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)
simple_rnn_model.fit(tmp_x, preproc_french_sentences_train, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 88230 samples, validate on 22058 samples
Epoch 1/10
88230/88230 [==============================] - 9s 99us/step - loss: 1.7738 - acc: 0.5713 - val_loss: 1.2766 - val_acc: 0.6338
Epoch 2/10
88230/88230 [==============================] - 8s 91us/step - loss: 1.1510 - acc: 0.6540 - val_loss: 1.0910 - val_acc: 0.6561
Epoch 3/10
88230/88230 [==============================] - 8s 90us/step - loss: 1.0160 - acc: 0.6743 - val_loss: 0.9782 - val_acc: 0.6820
Epoch 4/10
88230/88230 [==============================] - 8s 91us/step - loss: 0.9422 - acc: 0.6888 - val_loss: 0.9121 - val_acc: 0.6973
Epoch 5/10
88230/88230 [==============================] - 8s 91us/step - loss: 0.8803 - acc: 0.7050 - val_loss: 0.9311 - val_acc: 0.6768
Epoch 6/10
88230/88230 [==============================] - 8s 90us/step - loss: 0.8592 - acc: 0.7017 - val_loss: 0.8489 - val_acc: 0.7080
Epoch 7/10
88230/88230 [==============================] - 8s 90us/step - loss: 0.8192 - acc: 0.7131 - val_loss: 0.7996 - val_acc

### Model 2: Embedding - Training Set

In [20]:
# TODO: Reshape the input
# pad english sentences with max french sentence length
# (don't need to reshape into 3D array, since the embedding layer 
# does that for us)
tmp_x = pad(preproc_english_sentences_train, max_french_sequence_length)

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

# print the network layers
print(embed_rnn_model.summary())

embed_rnn_model.fit(tmp_x, preproc_french_sentences_train, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21)                0         
_________________________________________________________________
embed_layer (Embedding)      (None, 21, 256)           50944     
_________________________________________________________________
gru_layer (GRU)              (None, 21, 256)           393984    
_________________________________________________________________
td_layer (TimeDistributed)   (None, 21, 345)           88665     
Total params: 533,593
Trainable params: 533,593
Non-trainable params: 0
_________________________________________________________________
None
Train on 88230 samples, validate on 22058 samples
Epoch 1/10
88230/88230 [==============================] - 11s 125us/step - loss: 1.5516 - acc: 0.6708 - val_loss: 0.5179 - val_acc: 0.8393
Epoch 2/10
88230/88230 [==============================] - 10s 118us/step -

### Model 3: Bidirectional RNNs - Training Set

In [21]:
# pad english sentences with max french sentence length
tmp_x = pad(preproc_english_sentences_train, max_french_sequence_length)
# reshape english sentences into a 3D array required for the GRU layer
tmp_x = tmp_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

# TODO: Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

# print the network layers
print(bd_rnn_model.summary())
plot_model(bd_rnn_model, to_file='bd_rnn_plot.png', show_shapes=True, show_layer_names=True)

bd_rnn_model.fit(tmp_x, preproc_french_sentences_train, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21, 1)             0         
_________________________________________________________________
bd_layer (Bidirectional)     (None, 21, 512)           396288    
_________________________________________________________________
td_layer (TimeDistributed)   (None, 21, 345)           176985    
Total params: 573,273
Trainable params: 573,273
Non-trainable params: 0
_________________________________________________________________
None
Train on 88230 samples, validate on 22058 samples
Epoch 1/10
88230/88230 [==============================] - 15s 167us/step - loss: 1.5597 - acc: 0.6065 - val_loss: 1.1484 - val_acc: 0.6554
Epoch 2/10
88230/88230 [==============================] - 14s 156us/step - loss: 1.0418 - acc: 0.6785 - val_loss: 0.9724 - val_acc: 0.6914
Epoch 3/10
88230/88230 [==============================] - 14s 156us

### Model 4: Encoder Decoder - Training Set

In [22]:
# Reshaping the input to work with a basic RNN
# pad english sentences with max english sentence length
tmp_x = pad(preproc_english_sentences_train, max_english_sequence_length)
# reshape english sentences into a 3D array required for the GRU layer
tmp_x = tmp_x.reshape((preproc_english_sentences_train.shape[0], preproc_english_sentences_train.shape[1], 1))

# shift decoder input by one timestep
decoder_target_data = np.zeros(preproc_french_sentences_train.shape) 
decoder_target_data[:,:-1,:] = preproc_french_sentences_train[:,1:,:]

# create a tuple of the encoder and decoder input shapes
encode_decode_shape = (tmp_x.shape, decoder_target_data.shape)

# TODO: Train the neural network
encdec_rnn_model = encdec_model(
    encode_decode_shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

# print the network layers
print(encdec_rnn_model.summary())

encdec_rnn_model.fit([tmp_x, preproc_french_sentences_train], decoder_target_data, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict([tmp_x[:1], preproc_french_sentences_train[:1]])[0], french_tokenizer))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, 15, 1)        0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (None, 21, 1)        0                                            
__________________________________________________________________________________________________
encoder_gru (GRU)               [(None, 256), (None, 198144      encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_gru (GRU)               [(None, 21, 256), (N 198144      decoder_inputs[0][0]             
                                                                 encoder_gru[0][1]                
__________

### Model 5: Custom - Training Set

In [23]:
# TODO: Reshape the input
# pad english sentences with max french sentence length
# (don't need to reshape into 3D array, since the embedding layer 
# does that for us)
tmp_x = pad(preproc_english_sentences_train, max_french_sequence_length)

# TODO: Train the neural network
final_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

# print the network layers
print(final_model.summary())

final_model.fit(tmp_x, preproc_french_sentences_train, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(final_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 21)                0         
_________________________________________________________________
embed_layer (Embedding)      (None, 21, 256)           50944     
_________________________________________________________________
bd_layer (Bidirectional)     (None, 21, 1024)          2362368   
_________________________________________________________________
td_layer (TimeDistributed)   (None, 21, 345)           353625    
Total params: 2,766,937
Trainable params: 2,766,937
Non-trainable params: 0
_________________________________________________________________
None
Train on 88230 samples, validate on 22058 samples
Epoch 1/10
88230/88230 [==============================] - 34s 386us/step - loss: 1.7230 - acc: 0.6568 - val_loss: 0.5097 - val_acc: 0.8463
Epoch 2/10
88230/88230 [==============================] - 33s 371us/st

### Final Implementation


In [24]:
final_predictions(preproc_english_sentences_test, preproc_french_sentences_test, english_tokenizer, french_tokenizer)

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
il aime les poires les raisins et les pêches <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
il aime les poires les raisins et les pêches <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
